<a href="https://colab.research.google.com/github/merveenoyan/smol-vision/blob/main/inference_gists/InternVL3_Gist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## InternVL3

InternVL3 is a cutting edge vision language model based on InternViT and Qwen2.5 large language model. It comes in [a lot of sizes](https://huggingface.co/collections/merve/internvl3-hf-6814be2943b2ae0e711c92a5) and you can easily use all with Hugging Face transformers. We'll try 1B model here, it's pretty capable.

Let's see how to use it. Make sure we have the transformers in main as it was recently merged.

In [ ]:
!pip install -q git+https://github.com/huggingface/transformers.git

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


First, load the model and the processor.

In [ ]:
from transformers import AutoProcessor, AutoModelForImageTextToText, BitsAndBytesConfig
import torch

torch_device = "cuda"
model_checkpoint = "OpenGVLab/InternVL3-1B-hf"
processor = AutoProcessor.from_pretrained(model_checkpoint)
model = AutoModelForImageTextToText.from_pretrained(model_checkpoint, device_map=torch_device, torch_dtype=torch.bfloat16)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


processor_config.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

chat_template.jinja:   0%|          | 0.00/481 [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/666 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/6.86k [00:00<?, ?B/s]

...

added_tokens.json:   0%|          | 0.00/811 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/877 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.96k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.88G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/126 [00:00<?, ?B/s]

In [ ]:

messages = [
        {
            "role": "user",
            "content": [
                {"type": "image", "url": "https://huggingface.co/datasets/merve/vlm_test_images/resolve/main/throughput_smolvlm.png"},
                 {"type": "text", "text": "What is this chart about?"},
                ],
            },
        ]

inputs = processor.apply_chat_template(
    messages,
    padding=True,
    add_generation_prompt=True,
    tokenize=True,
    return_dict=True,
    return_tensors="pt",
).to(model.device, dtype=torch.bfloat16)

outputs = model.generate(**inputs, max_new_tokens=25)

decoded_outputs = processor.batch_decode(outputs, skip_special_tokens=True)
decoded_outputs

Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.


['user\n\nWhat is this chart about?\nassistant\nThis chart compares the throughput of three different models for generating text with a batch size of 1 on an H100']

You can also use this model for text-only inference.

In [ ]:
messages = [
    {
        "role": "user",
        "content": [
            {"type": "text", "text": "Write a joke about Turkish delight"},
        ],
    }
]

inputs = processor.apply_chat_template(messages, add_generation_prompt=True, tokenize=True, return_dict=True, return_tensors="pt").to(torch_device, dtype=torch.bfloat16)

generate_ids = model.generate(**inputs, max_new_tokens=50)
decoded_output = processor.decode(generate_ids[0, inputs["input_ids"].shape[1] :], skip_special_tokens=True)

print(decoded_output)

Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.


Why did the Turkish delight go to the doctor?

Because it had a sweet tooth and needed a dental check-up!
